# Importing necessary libraries to run models

In [22]:
import numpy as np
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from pickle import load
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

# Need to load my 5 txt files to make the 5 models

In [23]:
def loader(textfile):
    # Open the file as read-only
    file = open(textfile, 'r')
    # Actually read the file
    text = file.read()
    # Now that I have the data in my text variable, need to close out of the file
    file.close()
    return text

In [37]:
# Create the 5 instances of the 5 ratings

one = loader('One_Star.txt')
# two = loader('Two_Star.txt')
# three = loader('Three_Star.txt')
# four = loader('Four_Star.txt')
# five = loader('Five_Star.txt')

In [25]:
# Need to split up each of these data sets into training sequences by the different lines. 
# End goal is to have user input 5 words and the text generator will work its magic

In [38]:
one_lines = one.split('\n')
# two_lines = two.split('\n')
# three_lines = three.split('\n')
# four_lines = four.split('\n')/
# five_lines = five.split('\n')

In [39]:
# Each line has been broken out with 5 sequence of words
one_lines[:5]

['atrocious the live music was loud',
 'the live music was loud and',
 'live music was loud and mediocre',
 'music was loud and mediocre the',
 'was loud and mediocre the decor']

# Need to convert the words in my data to integers in order to be used by the model. 

Will need to train the keras tokenizer on entire dataset to assign a unique ID to each unique word in the dataset

In [40]:
tokenizer = Tokenizer()

def word_to_int(lines):
    tokenizer.fit_on_texts(lines)
    sequences = tokenizer.texts_to_sequences(one_lines)
    # Need the total size of the vocabulary for our embedding layer in model
    vocab_size = len(tokenizer.word_index) + 1
    return sequences, vocab_size

In [41]:
# Running the function for all 5 data sets
one_sequence, one_vocab_size = word_to_int(one_lines)
# two_sequence, two_vocab_size = word_to_int(two_lines)
# three_sequence, three_vocab_size = word_to_int(three_lines)
# four_sequence, four_vocab_size = word_to_int(four_lines)
# five_sequence, five_vocab_size = word_to_int(five_lines)

In [42]:
# each line has been converted to unique ints to represent each word
len(one_sequence[0])

6

In [14]:
part_one = one_sequence[0:200000]

part_two = one_sequence[200001:400000]

part_three = one_sequence[400001:600000]

part_four = one_sequence[600001:800000]

part_five = one_sequence[800001:1000000]

part_six = one_sequence[1000001:1200000]

part_seven = one_sequence[1200001:-1]

# Need to convert my sequences into inputs and outputs to structure my model to predict words based on previous set of words

## One Star Review Modeling

In [43]:
one_sequences = np.array(one_sequence)


In [44]:
one_sequences

array([[ 2374,     1,   551,   549,     7,   809],
       [    1,   551,   549,     7,   809,     3],
       [  551,   549,     7,   809,     3,  1044],
       ...,
       [ 5128,  4040,   284,    32,  2091, 12323],
       [ 4040,   284,    32,  2091, 12323,     8],
       [  284,    32,  2091, 12323,     8,    15]])

In [36]:
X, y = one_sequence[:,:-1], one_sequence[:,-1]
y = to_categorical(y, num_classes=one_vocab_size)
seq_length = X.shape[1]

IndexError: too many indices for array

In [18]:
# defining my model

model = Sequential()
model.add(Embedding(one_vocab_size, 50, input_length=seq_length))
model.add(LSTM(25, return_sequences=True))
model.add(LSTM(25))
model.add(Dense(25, activation='relu'))
model.add(Dense(one_vocab_size, activation = 'softmax'))
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 5, 50)             1883750   
_________________________________________________________________
lstm_1 (LSTM)                (None, 5, 25)             7600      
_________________________________________________________________
lstm_2 (LSTM)                (None, 25)                5100      
_________________________________________________________________
dense_1 (Dense)              (None, 25)                650       
_________________________________________________________________
dense_2 (Dense)              (None, 37675)             979550    
Total params: 2,876,650
Trainable params: 2,876,650
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [24]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X,y,batch_size=256, epochs = 25)

Epoch 1/25
1370405/1370405 [==============================] - 1501s 1ms/step - loss: 6.3984 - acc: 0.0792
Epoch 2/25
1370405/1370405 [==============================] - 1549s 1ms/step - loss: 5.9729 - acc: 0.1117
Epoch 3/25
1370405/1370405 [==============================] - 1619s 1ms/step - loss: 5.8094 - acc: 0.1222
Epoch 4/25
1370405/1370405 [==============================] - 1648s 1ms/step - loss: 5.6973 - acc: 0.1293
Epoch 5/25
1370405/1370405 [==============================] - 1647s 1ms/step - loss: 5.6134 - acc: 0.1345
Epoch 6/25
1370405/1370405 [==============================] - 1683s 1ms/step - loss: 5.5464 - acc: 0.1389
Epoch 7/25
1370405/1370405 [==============================] - 1673s 1ms/step - loss: 5.4907 - acc: 0.1425
Epoch 8/25
1370405/1370405 [==============================] - 1666s 1ms/step - loss: 5.4425 - acc: 0.1460
Epoch 9/25
1370405/1370405 [==============================] - 1652s 1ms/step - loss: 5.4006 - acc: 0.1488
Epoch 10/25
1370405/1370405 [=================

In [25]:
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))